### Introduction
The aim of this notebook is to ensure all books belong to either a `Fiction` or `Nonfiction` category. There are different book categories in the dataset. Here is the approach I planned to follow:
* An category that contains the word 'fiction' will be classified as `Fiction`.
* An category that contains the word 'nonfiction' will be classified as `Nonfiction`.
* Every other categories will be labeled as either `Fiction` or `Nonfiction` based on a pretrained classification model from Huggingface.

In [4]:
from tqdm import tqdm


import numpy as np
import pandas as pd

from transformers import pipeline

from dotenv import load_dotenv

# import warnings
# warnings.filterwarnings("ignore")

In [5]:
load_dotenv()

True

### Data Loading

Loading the cleaned dataset that was saved form the `data_exploration_cleaning_analysis.ipynb` notebook.

In [6]:
books = pd.read_csv("data/books_cleaned.csv")

In [6]:
books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5693 entries, 0 to 5692
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   isbn13              5693 non-null   int64  
 1   isbn10              5693 non-null   object 
 2   title               5693 non-null   object 
 3   authors             5646 non-null   object 
 4   categories          5662 non-null   object 
 5   thumbnail           5506 non-null   object 
 6   description         5693 non-null   object 
 7   published_year      5693 non-null   float64
 8   average_rating      5693 non-null   float64
 9   num_pages           5693 non-null   float64
 10  ratings_count       5693 non-null   float64
 11  title_and_subtitle  5693 non-null   object 
 12  tagged_description  5693 non-null   object 
dtypes: float64(4), int64(1), object(8)
memory usage: 578.3+ KB


### Exploring the `categories` Column

I would be creating a new column that containt two (2) unique values: `Fiction` and `Nonfiction`.

The top 20 most popular categories are:

In [7]:
popular_categories = books["categories"].value_counts().reset_index()
print(f"There are {popular_categories.shape[0]} unique book categories")

print(f"\nThe first 20 most popular books are:-")
popular_categories[:20]

There are 499 unique book categories

The first 20 most popular books are:-


,categories,count
0,Fiction,2269
1,Juvenile Fiction,456
2,Biography & Autobiography,348
3,History,226
4,Literary Criticism,138
5,Philosophy,129
6,Religion,126
7,Comics & Graphic Novels,125
8,Drama,99
9,Juvenile Nonfiction,81


Now, examine all categories that contains the word fiction and nonficition:

In [8]:
fictional_books = books[books["categories"].str.lower().str.contains("fiction", na=False)]
print(f"The total number of fictional books recorded are: {fictional_books.shape[0]}.")
print(f"\nThese categories are:")
fictional_categories = fictional_books.categories.unique()
print(fictional_categories)

The total number of fictional books recorded are: 2902.

These categories are:
['Fiction' 'Fantasy fiction' 'English fiction' 'Science fiction'
 'American fiction' 'Juvenile Fiction' 'Fantasy fiction, English'
 'FICTION' 'Juvenile Nonfiction' 'Young Adult Fiction'
 'Australian fiction' 'Domestic fiction' 'Classical fiction'
 'Historical fiction' 'Experimental fiction, American'
 'Experimental fiction' 'Diary fiction' 'Adventure fiction'
 'Political fiction' 'Occult fiction' 'Arabic fiction' 'Czech fiction'
 'Alternative histories (Fiction)' 'Boarding school-fiction'
 'Fantasy fiction, American' 'Humorous fiction' 'JUVENILE FICTION'
 'European fiction' 'Science fiction, American' 'Christian fiction'
 'Indic fiction (English)']


In [9]:
non_fictional_books = books[books["categories"].str.lower().str.contains("nonfiction", na=False)]
print(f"The total number of nonfictional books recorded are: {non_fictional_books.shape[0]}.")
print(f"\nThese categories are:")
non_fictional_categories = non_fictional_books.categories.unique()
print(non_fictional_categories)

The total number of nonfictional books recorded are: 81.

These categories are:
['Juvenile Nonfiction']


Some of the columns form the most popular categories do not contain the word `fiction` or `nonfiction` but it is obvious they can fictional or non-fictional. Recreating the new column:

In [10]:
# names of the popular categories:
popular_categories.categories[:20].unique()

array(['Fiction', 'Juvenile Fiction', 'Biography & Autobiography',
       'History', 'Literary Criticism', 'Philosophy', 'Religion',
       'Comics & Graphic Novels', 'Drama', 'Juvenile Nonfiction',
       'Science', 'Poetry', 'Business & Economics',
       'Literary Collections', 'Social Science', 'Performing Arts',
       'Body, Mind & Spirit', 'Art', 'Travel', 'Cooking'], dtype=object)

In [11]:
fictional_category_mapping = {fictional_category:"Fiction" for fictional_category in fictional_categories}

non_fictional_category_mapping = {non_fictional_category:"Nonfiction" for non_fictional_category in non_fictional_categories}

other_category_mapping = {
 'Biography & Autobiography': "Nonfiction",
 'History': "Nonfiction",
 'Literary Criticism': "Nonfiction",
 'Philosophy': "Nonfiction",
 'Religion': "Nonfiction",
 'Comics & Graphic Novels': "Fiction",
 'Drama': "Fiction",
 'Science': "Nonfiction",
 'Poetry': "Fiction"}

# combine all categories together
category_mapping = {**fictional_category_mapping, **non_fictional_category_mapping, **other_category_mapping}

In [12]:
books["simple_categories"] = books["categories"].map(category_mapping)

In [13]:
books.head(2)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN


Books that are note in category_mapping will be assigned `NaN`. 

Examining categories that are not `NaN`:

In [14]:
books[~(books["simple_categories"].isna())]

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
8,9780006482079,0006482074,Warhost of Vastmark,Janny Wurts,Fiction,http://books.google.com/books/content?id=uOL0f...,"Tricked once more by his wily half-brother, Ly...",1995.0,4.03,522.0,2966.0,Warhost of Vastmark,9780006482079 Tricked once more by his wily ha...,Fiction
11,9780006483908,0006483909,Jimmy the Hand,Raymond E. Feist;S. M. Stirling,Fantasy fiction,http://books.google.com/books/content?id=hV4-o...,"Jimmy the Hand, boy thief of Krondor, lived in...",2003.0,3.95,368.0,5579.0,Jimmy the Hand,"9780006483908 Jimmy the Hand, boy thief of Kro...",Fiction
15,9780006496878,0006496873,Mystical Paths,Susan Howatch,English fiction,http://books.google.com/books/content?id=by4yt...,1968 finds Nicholas Darrow wrestling with pers...,1996.0,4.23,576.0,1023.0,Mystical Paths,9780006496878 1968 finds Nicholas Darrow wrest...,Fiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5686,9788171565641,8171565646,Aspects of the Novel,E. M. Forster,English fiction,http://books.google.com/books/content?id=qWU9P...,"Forster's lively, informed originality and wit...",2004.0,3.83,141.0,10.0,Aspects of the Novel,"9788171565641 Forster's lively, informed origi...",Fiction
5687,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction
5690,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Nonfiction
5691,9789027712059,9027712050,The Berlin Phenomenology,Georg Wilhelm Friedrich Hegel,History,http://books.google.com/books/content?id=Vy7Sk...,Since the three volume edition ofHegel's Philo...,1981.0,0.00,210.0,0.0,The Berlin Phenomenology,9789027712059 Since the three volume edition o...,Nonfiction


There were `4213` books labeled as either `Fiction` or `Nonfiction` out of a total of `5693` recorded books.

### Zero-Shot Classifier Model

Huggingface zero-shot classifier pre-trained model will be used to classify book as either `Fictional` or `Nonfictional`.

In [16]:
# the 2 categories we want to use in our model:
fiction_categories = ["Fiction", "Nonfiction"]

pipe = pipeline("zero-shot-classification",
                model="facebook/bart-large-mnli",
                device="cpu")

C:\Users\DELL\Desktop\New-Era-Projects\Movie-Recommendation-System\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\DELL\.cache\huggingface\hub\models--facebook--bart-large-mnli. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


#### Classification Inference

In [20]:
# the descrioction of the first fictional book
sequence = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)[0]
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

**Predict the probability of the book description above of being fictional or nonfictiona**

In [22]:
inference = pipe(sequence, fiction_categories)
inference

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [25]:
# obtain the label of the score with highest probability
max_index = np.argmax(inference["scores"])
max_label = inference["labels"][max_index]
max_label

'Fiction'

In [30]:
def predict_category(sequence: str, categories: list) -> str:
    """"
    Predicts if a sequence belong to any categories specified in categories.
    """
    predictions = pipe(sequence, categories)
    max_index = np.argmax(predictions["scores"])
    max_label = predictions["labels"][max_index]
    return max_label

### Evaluating the Classification


#### Predicting Fictional and Nonfictional Books
How well does the model predicts fictional books (first 300 fictional books)?

In [31]:
actual_cats = []
predicted_cats = []

fiction_books = books.loc[books["simple_categories"] == "Fiction", "description"].reset_index(drop=True)

for i in tqdm(range(min(300, len(fiction_books)))):
    sequence = fiction_books[i]
    predicted_cats.append(predict_category(sequence, fiction_categories))
    actual_cats.append("Fiction")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [20:20<00:00,  4.07s/it]


How well does the model predicts nonfictional books (first 100 nonfictional books?

In [36]:
nonfiction_books = books.loc[books["simple_categories"] == "Nonfiction", "description"].reset_index(drop=True)

for i in tqdm(range(min(100, len(nonfiction_books)))):
    sequence = nonfiction_books[i]
    predicted_cats.append(predict_category(sequence, fiction_categories))
    actual_cats.append("Nonfiction")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [08:10<00:00,  4.91s/it]


In [37]:
predictions_df = pd.DataFrame({"actual_categories": actual_cats, "predicted_categories": predicted_cats})
predictions_df.tail()

,actual_categories,predicted_categories
398,Nonfiction,Nonfiction
399,Nonfiction,Nonfiction
400,Nonfiction,Nonfiction
401,Nonfiction,Nonfiction
402,Nonfiction,Nonfiction


In [38]:
predictions_df["correct_prediction"] = (
    np.where(predictions_df["actual_categories"] == predictions_df["predicted_categories"], 1, 0)
)

In [39]:
# accuracy score
predictions_df["correct_prediction"].sum() / len(predictions_df)

np.float64(0.6923076923076923)

#### Predicting books with missing categories

Books with unknown categories:

In [56]:
missing_cats = books.loc[books["simple_categories"].isna(), ["isbn13", "description"]].reset_index(drop=True) 
print(f"Total number of books with unknown categories: {missing_cats.shape[0]}")

Total number of books with unknown categories: 1480


Predict if any of the books with unknown category as either `Fiction` or `Nonfiction`

In [57]:
isbns = []
predicted_cats = []

for i in tqdm(range(0, len(missing_cats))):
    sequence = missing_cats["description"][i]
    predicted_cats.append(predict_category(sequence, fiction_categories))
    isbns.append(missing_cats["isbn13"][i])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1480/1480 [1:24:04<00:00,  3.41s/it]


In [59]:
missing_predicted_df = pd.DataFrame({"isbn13": isbns, "predicted_categories": predicted_cats})
missing_predicted_df.head()

,isbn13,predicted_categories
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction


> Merge the `missing_predicted_df` dataframe `predicted_categories` column to the `books` dataframe `simple_categories` column based on similar `isbn13` column.

In [71]:
# updated books dataframe:
books = pd.merge(books, missing_predicted_df, on="isbn13", how="left")

In [72]:
books["simple_categories"] = np.where(books["simple_categories"].isna(), books["predicted_categories"], books["simple_categories"])

# remove the predicted_categories
books.drop(columns = ["predicted_categories"], inplace=True)

In [74]:
books.head(3)

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction


### Save the Newly Created Dataframe

Now the I have labeled all books to belong to either `Fiction` and `Nonfiction` category, I would save the new dataframe as csv to avoid making inference in labeling the books in the dataset again:

In [75]:
books.to_csv("data/books_with_categories.csv", index=False)